In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:

from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

In [3]:
from fastai.text import *
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/root/.fastai/data/imdb_sample/texts.csv')]

In [6]:
df = pd.read_csv(path/'texts.csv')
df.head(-10)

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False
...,...,...,...
985,negative,Don't bother. A little prosciutto could go a l...,True
986,negative,I found the characters mediocre and the story ...,True
987,positive,"The pre-release version of 1933's ""Baby Face"" ...",True
988,negative,By 1976 the western was an exhausted genre and...,True


In [13]:
df['text'][:10]

0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
5    From the start, you know how this movie will e...
6    There were a lot of truly great horror movies ...
7    I was fortunate enough to meet George Pal (and...
8    This film is the freshman effort of Stephanie ...
9    Greg Davis and Bryan Daly take some crazed sta...
Name: text, dtype: object

In [18]:
df['text'][1]

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [0]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [0]:
data_lm.save()

In [0]:
data = load_data(path)

In [25]:
data = TextClasDataBunch.from_csv(path, 'texts.csv')
data.show_batch(10)

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj homer xxmaj hickam as they did in the film adaptation of his book , xxmaj rocket xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive
"xxbos xxmaj the trouble with the book , "" xxmaj memoirs of a xxmaj geisha "" is that it had xxmaj japanese surfaces but underneath the surfaces it was all an xxmaj american man 's way of thinking . xxmaj reading the book is like watching a magnificent ballet with great music , sets , and costumes yet performed by xxunk animals dressed in those xxunk far from xxmaj japanese",negative
"xxbos xxmaj the xxmaj blob starts with one of the most bizarre theme songs ever , sung by an uncredited xxmaj burt xxmaj xxunk of all people ! xxmaj you really have to hear it to believe it , xxmaj the xxmaj blob may be worth watching just for this song alone & my user comment summary is just a little taste of the classy lyrics ... xxmaj after this",negative
"xxbos xxmaj well , what can i say . \n \n "" xxmaj what the xxmaj bleep do we xxmaj know "" has achieved the nearly impossible - leaving behind such masterpieces of the genre as "" xxmaj the xxmaj postman "" , "" xxmaj the xxmaj dungeon xxmaj master "" , "" xxmaj xxunk "" , and so fourth , it will go down in history as the",negative
"xxbos xxmaj bonanza had a great cast of wonderful actors . xxmaj lorne xxmaj greene , xxmaj pernell xxmaj whitaker , xxmaj michael xxmaj landon , xxmaj dan xxmaj blocker , and even xxmaj guy xxmaj williams ( as the cousin who was brought in for several episodes during 1964 to replace xxmaj adam when he was leaving the series ) . xxmaj the cast had chemistry , and they",positive


In [26]:
data.vocab.itos[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']

In [27]:
data.train_ds[0][0]

Text xxbos i came at this film with high expectations . i was aware of xxmaj xxunk 's work and ' xxmaj the xxmaj xxunk ' and was interested in an adaptation . i first wanted to switch off after ten minutes , but felt that it would be unfair . xxmaj there was a representation of a storm , but where in your mind do you xxunk a small boy peeing over a toy boat ? xxmaj it is symbolic of what ? i continued another ten minutes my finger xxunk over the ' off ' button , somewhere something would capture my interest . xxmaj this is not xxmaj shakespeare , it is not cinema . xxmaj there is a time and place for it , but i will not waste my time and there is no place for it in my studies of xxmaj shakespeare . xxmaj after twenty five minutes i gave up and that was the end . i then read all the comments on this website and the pretentiousness of the film is only matched by its defendants . ' xxmaj its a painting xxrep 5 . then put it in a gallery ' , ' it 's a ballet xxrep 5 . keep it on the stage 

In [28]:

data.train_ds[0][0].data[:10]

array([   2,   19,  411,   46,   21,   32,   30,  331, 1102,   10])

In [0]:

data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

In [0]:
bs=48

In [31]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/root/.fastai/data/imdb/README'),
 PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/unsup')]

In [32]:
(path/'train').ls()

[PosixPath('/root/.fastai/data/imdb/train/labeledBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/pos'),
 PosixPath('/root/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/neg')]

In [0]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.2)#!
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')

In [0]:
data_lm = load_data(path, 'data_lm.pkl', bs=bs)

In [37]:
data_lm.show_batch(20)

idx,text
0,"xxmaj the xxmaj downfall "" as you might now it ) which are also very realistic . \n \n xxmaj one of those movies is "" xxmaj rosenstrasse "" . xxmaj it tells a true story and deals with the subject of the mixed marriages during the war , even though the movie starts with a family in the xxup usa , at the present day . xxmaj after"
1,"those so inclined , yes , he does get naked ) whose performance is not only highly believable , but is done with such skill that his xxmaj eastwood - esquire character is both bad - to - the - bone and likable ( a very delicate mix ) . xxmaj add him to a cast of ' hey - wait - a - minute - i - know -"
2,"to be flogged with yet another xxmaj american xxmaj pie film xxup and another xxmaj scary xxmaj movie ( which are only scary because of their sheer barbarous idiocy ) , it 's nice to see that there are still people making comedies worth watching . xxmaj do n't miss this one . xxbos i 've been a fan of xxmaj xu xxmaj ke ( xxmaj hark xxmaj tsui )"
3,"a xxmaj nazi you love . :) xxbos xxmaj no , this hilariously horrible 70 's made - for - xxup tv horror clinker is n't about a deadly demonically possessed dessert cake . xxmaj still , this exceptionally awful , yet undeniably amusing and thus enjoyable cathode ray refuse reaches a breathtaking apex of absolute , unremitting silliness and xxunk that 's quite tasty in a so - execrable"
4,"xxmaj halo at a point ) and xxmaj andrew 's amazing guitar solo , among many other things . \n \n xxmaj all in all a great feel - good film about friendship . xxmaj you have to see this ! xxbos xxup the xxup great xxup caruso was the biggest hit in the world in 1951 and broke all box office records at xxmaj radio xxmaj city xxmaj"
5,"xxmaj columbia xxmaj records for four years . xxmaj kelly was already well - known as a dancer , but xxmaj iturbi had by then been a world - wide sensation for 20 + years . \n \n xxmaj and as to the lack of a plot , xxmaj americans did n't need plot . xxmaj they were tired of war , they were sick with fear for their"
6,you owe it to yourself to see it . xxbos xxmaj after seeing xxmaj arthur on xxup tv numerous times i laughed so hard xxmaj dudley xxmaj moore 's role as xxmaj arthur a happy go luck drunken millionaire was hysterical but xxmaj john xxmaj gielgud as his servant / father figure xxmaj hobson was equally charming . i bought the xxup dvd a few weeks ago and watched it
7,"less -- including an over - reliance on stock actors that were seen again and again , no money for re - shooting scenes and a need to get the films done xxup fast ! xxmaj this film tried very hard to be a xxmaj black version of a xxmaj gene xxmaj autry film -- starring xxmaj herb xxmaj jeffries instead . xxmaj jeffries was a light - skinned man"
8,", another political thriller , that bored me to tears . xxmaj it 's too old , too tired , and most importantly the lack of effort sucked . xxmaj not recommended . \n \n 4 / 10 xxbos i was expecting the movie based on xxmaj grendel , the book written by xxmaj john xxmaj gardner in the late 1970 's . xxmaj it was based on the"
9,"only fair to say that my opinion of the director and his previous work is as low as it is possible to be but i am happy to point out that there are a few elements that boarder on pleasurable and are a great improvement on his previous film , xxmaj darkhunters , which is one of the worst films i have ever seen . xxmaj at times the cinematography"


In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.2)#3

In [0]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learn.recorder.plot(skip_end=15)

In [0]:

learn.fit_one_cycle(3, 1e-2, moms=(0.8,0.7))

In [0]:
learn.save('fit_head')

In [0]:
learn.load('fit_head');

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [0]:
learn.save('fine_tuned')

In [0]:
learn.load('fine_tuned');

In [0]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [0]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [0]:
learn.save_encoder('fine_tuned_enc')

In [0]:
path = untar_data(URLs.IMDB)

In [0]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))

data_clas.save('data_clas.pkl')

In [0]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [0]:
data_clas.show_batch()

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.4)
learn.load_encoder('fine_tuned_enc')

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(3, 2e-2, moms=(0.8,0.7))

In [0]:
learn.save('first')

In [0]:

learn.load('first');

In [0]:

learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [0]:
learn.save('second')

In [0]:
learn.load('second');

In [0]:
learn.freeze_to(-3)
learn.fit_one_cycle(5, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [0]:
learn.save('third')

In [0]:
learn.load('third');

In [0]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [0]:
learn.predict("I really loved that movie, it was awesome!")